In [1]:
# Python Package imports
import numpy as np
import pandas as pd
import glob
import scipy.misc
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures


In [2]:
# Maximum number of threads that will be spawned
MAX_THREADS = 30

In [3]:
movie_title_arr = []
movie_year_arr = []
movie_genre_arr = []
movie_synopsis_arr =[]
image_url_arr  = []
image_id_arr = []

In [4]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText()
    except:
        return 'NA'

def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText()
    except:
        return 'NA'


def getImage(image):
    try:
        return image.get('loadlate')
    except:
        return 'NA'

def getImageId(image):
    try:
        return image.get('data-tconst')
    except:
        return 'NA'

In [5]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for movie in movies_list:
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        #  Movie Title
        movie_title =  getMovieTitle(header)
        movie_title_arr.append(movie_title)
        
        
        #  Genre  of movie
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        
        #  Image attributes
        img_url = getImage(image)
        image_url_arr.append(img_url)
        
        image_id = image.get('data-tconst')
        image_id_arr.append(image_id)

In [6]:
# An array to store all the URL that are being queried
imageArr = []

# Maximum number of pages one wants to iterate over
MAX_PAGE =30

# Loop to generate all the URLS.
for i in range(0,MAX_PAGE):
    totalRecords = 0 if i==0 else (250*i)+1
    imdb_url = f'https://www.imdb.com/search/title/?release_date=2020-01-02,2021-02-01&user_rating=4.0,10.0&languages=en&count=250&start={totalRecords}&ref_=adv_nxt'
    imageArr.append(imdb_url)

In [7]:
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(main, story_urls)

In [8]:
# Call the download function with the array of URLS called imageArr
download_stories(imageArr)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Genre": movie_genre_arr,
    "image_url": image_url_arr,
    "image_id": image_id_arr,
})

print('--------- Download Complete CSV Formed --------')

movieDf.to_csv('movies_data_test.csv', index=False)
movieDf.head()

--------- Download Complete CSV Formed --------


,Title,Genre,image_url,image_id
0,Raised by Wolves,"\nDrama, Fantasy, Sci-Fi",https://m.media-amazon.com/images/M/MV5BNzk0ZT...,tt9170108
1,Ted Lasso,"\nComedy, Drama, Sport",https://m.media-amazon.com/images/M/MV5BMDVmOD...,tt10986410
2,Snowpiercer,"\nAction, Drama, Sci-Fi",https://m.media-amazon.com/images/M/MV5BYTY1MD...,tt6156584
3,À l'ombre des magnolias,"\nDrama, Romance",https://m.media-amazon.com/images/M/MV5BZjBmNT...,tt9077540
4,Clean,"\nCrime, Drama",https://m.media-amazon.com/images/M/MV5BZGZiOD...,tt10023022


In [9]:
movies = pd.read_csv("movies_data_test.csv",encoding="ISO-8859-1")

#Print the label categories we are working with
print(movies.columns)


#Print the total amount of movies in the dataset
print("Total Number of Movies is: " + str(len(movies.index)))

#We will count the number of movies with a specific genre
genrelist = []
length = len(movies)
for n in range(0,length):
    genres = str(movies.loc[n]["Genre"])
    genres = genres.split(",")
    genrelist.extend(genres)
    
#We will also display number of Genres     
unique_genres = list(set(genrelist))
print("Total Number of Genres is: " + str(len(unique_genres)))

Index(['Title', 'Genre', 'image_url', 'image_id'], dtype='object')
Total Number of Movies is: 7500
Total Number of Genres is: 102


In [10]:
import requests
import time
import re
import os

def download_poster(downloaded_image_dir, title, label, poster_path):
  
    if not os.path.exists(downloaded_image_dir):
        os.makedirs(downloaded_image_dir)
        
    if not os.path.exists(downloaded_image_dir+'/'+label):
        os.makedirs(downloaded_image_dir+'/'+label)

    imgUrl = 'http://image.tmdb.org/t/p/w185/' + poster_path

    local_filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'

    try:
        session = requests.Session()
        r = session.get(imgUrl, stream=True, verify=False)
        with open(downloaded_image_dir+'/'+label+'/'+local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
    except:
        print('PROBLEM downloading', title,label,poster_path,imgUrl)
    
    time.sleep(1)

# download image by iterate pandas
for index, row in df_movies.iterrows():
    download_poster(
        'images_movies_genre',
        str(row['title']),
        str(row['genre']),
        row['poster_path']
    )

NameError: name 'df_movies' is not defined

In [ ]:
path = 'posters/'
data = pd.read_csv("movies_data_test.csv", encoding="ISO-8859-1")


image_glob = glob.glob(path + "/" + "*.jpg")
img_dict = {}


def get_id(filename):
    index_s = filename.rfind("/") + 1
    index_f = filename.rfind(".jpg")
    return filename[index_s:index_f]

for fn in image_glob:
    try:
        img_dict[get_id(fn)] = scipy.misc.imread(fn)
    except:
        pass

def show_img(id):
    title = data[data["image_id"] == int(id)]["Title"].values[0]
    genre = data[data["image_id"] == int(id)]["Genre"].values[0]
    plt.imshow(img_dict[id])
    plt.title("{} \n {}".format(title, genre))

show_img("2")